<a href="https://colab.research.google.com/github/CRISTIANJULIOCESAR/2022-1-iimas-deep/blob/main/XGBOOST_GPU_shaps_last_cv_and_all_better_separation_V100_GPU_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install shap==0.43.0 umap-learn==0.5.4

#Only tumor wo normal(healthy) preprocessing

In [3]:
import shap
import pandas as pd
import numpy as np
df =  pd.read_pickle('/content/drive/MyDrive/pan_cancer_diner/Gene parsing  code /11093rx39979c.pkl')
meta= pd.read_pickle('/content/drive/MyDrive/pan_cancer_diner/Clinical data from  tcga.R/metadata.pkl')

In [4]:
meta.columns

Index(['Unnamed: 0', 'data_type', 'updated_datetime', 'file_name', 'md5sum',
       'data_category', 'experimental_strategy', 'project', 'sample_uuid',
       'sample_barcode', 'tumor', 'patient_barcode', 'gender', 'vital',
       'days_to_contact', 'days_to_death', 'days_to_birth', 'panel',
       'histology', 'tissue_site', 'stage', 'T', 'N', 'M', 'residual_tumor',
       'new_tumor_events', 'follow_ups'],
      dtype='object')

In [5]:
df

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3,hsa-mir-1253,hsa-mir-423,snoZ196
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
84fd87d4-9b47-4852-b45c-1f681a58832c,4,0,949,48,74,2,27981,15,288,0,...,1,0,1057,4318,0,973,924,0,0,1
2da36252-af74-4e0c-ae38-5f5fc6bdad6e,0,0,285,10,41,4,33182,80,586,0,...,0,0,3719,5613,0,10839,1977,0,0,2
20ef7e72-9b76-4a4f-985c-1c35503b3e86,0,0,82,2,20,0,14847,115,122,0,...,185,0,2622,6686,0,2412,2116,0,0,4
83acd71c-c12a-4394-ba8d-05e9c5ad2cf1,1,0,502,44,278,9,50129,139,26,1,...,531,0,6497,9902,0,7520,6272,0,0,0
04d7ee0b-95ce-4af2-8140-547e9c6bd187,2,0,612,6,84,3,31641,55,518,0,...,8,0,2446,11845,0,4063,2797,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5d98ad4a-483f-4032-ae9e-0ab6d398da98,3,0,3455,81,216,1,23,4,10,0,...,2,0,907,28542,0,3221,1042,0,0,8
e0822da0-24d1-413a-b87f-5c6a242d3732,0,0,254,29,192,9,53226,74,38,1,...,126,0,2741,11978,0,5163,5026,0,0,5
3a74e5d3-48e7-4df9-85e1-dd02d1c804c1,1,0,95,135,674,0,41441,54,12,0,...,72,0,2296,27635,0,5424,2489,0,0,6


#Live is 0 Death is 1

In [6]:
df = df[meta['tumor'] == True]
df['vital']= meta['vital']
df =df.dropna()

Boolean Series key will be reindexed to match DataFrame index.

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df['vital'].unique()

array(['Alive', 'Dead'], dtype=object)

In [8]:
y = df['vital'].replace({'Alive': 1, 'Dead': 0})
y

Unnamed: 0
84fd87d4-9b47-4852-b45c-1f681a58832c    1
2da36252-af74-4e0c-ae38-5f5fc6bdad6e    1
83acd71c-c12a-4394-ba8d-05e9c5ad2cf1    1
04d7ee0b-95ce-4af2-8140-547e9c6bd187    1
171f45e7-83e7-4286-970a-0dafa6f46d8a    1
                                       ..
21ea6561-2bd1-4a3d-ae70-3c504530bb26    0
5d98ad4a-483f-4032-ae9e-0ab6d398da98    0
3a74e5d3-48e7-4df9-85e1-dd02d1c804c1    1
86fd69ec-fbe4-4ca7-a1a5-a42367527929    0
a811e4bd-d1d5-4a97-bc7f-899906c7f145    1
Name: vital, Length: 10339, dtype: int64

In [9]:
y.value_counts()


1    7251
0    3088
Name: vital, dtype: int64

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Load sample data
#Without the columun named "vital"
X = np.log(df.iloc[:,:-1].values +1)
y = df['vital'].replace({'Alive': 1, 'Dead': 0})
y = y.values

# Cross-validation setup
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
aucs = []

clf = XGBClassifier(
    n_estimators= 1000,
    learning_rate=0.01,
    max_depth=20,
    subsample=0.5,
    colsample_bytree=0.2,
    colsample_bylevel=0.2,
    gamma=2,
    reg_alpha=0.5,
    reg_lambda=2,
    eval_metric='auc',
    use_label_encoder=False,
    grow_policy='lossguide',
    verbosity=3,
    #tree_method='gpu_hist',
    n_jobs=-1,tree_method = "hist", device = "cuda"
)


"""
clf = XGBClassifier(
    n_estimators= 200,
    eval_metric='auc',
    verbosity=3,
    #tree_method='gpu_hist',
    random_state=50,
    grow_policy='lossguide',
    tree_method='gpu_hist',

)
"""

StratifiedKFold is a variation of k-fold cross-validation that returns stratified folds. "Stratified" means that each fold is made by preserving the percentage of samples for each class. This is especially useful when you have an imbalanced dataset where one class significantly outnumbers the other(s).

In [ ]:
# Create a figure for the ROC curves
plt.figure(figsize=(10, 8))
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)

    # Predict the class labels
    y_pred_class = clf.predict(X_test)

    # Print the classification report for this fold
    print(f"Classification Report for Fold {len(aucs) + 1}:")
    print(classification_report(y_test, y_pred_class))
    print("-" * 50)  # For visual separation between reports

    # Predict the probability of positive class
    y_pred = clf.predict_proba(X_test)[:, 1]

    # Compute and store AUC for the fold
    auc = roc_auc_score(y_test, y_pred)
    aucs.append(auc)

    # Compute ROC curve and plot
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr, label=f'AUC (Fold {len(aucs)}): {auc:.2f}')

# Plotting the ROC curves
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for each fold')
plt.legend(loc="lower right")
plt.show()
plt.savefig("ROC.png", dpi=600)  # dpi can be adjusted based on desired resolution

# Reporting average and standard deviation of AUC
print(f"Mean AUC: {np.mean(aucs):.2f}")
print(f"Std AUC: {np.std(aucs):.2f}")

In [ ]:
clf

**WITH ALL DATA** SHAPS

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Load sample data
#Without the columun named "vital"
X = np.log(df.iloc[:,:-1].values +1)
y = df['vital'].replace({'Alive': 1, 'Dead': 0})
y = y.values


In [ ]:
"""
clf = XGBClassifier(
    n_estimators= 100,
    eval_metric='auc',
    verbosity=3,
    #tree_method='gpu_hist',
    random_state=50,
    grow_policy='lossguide',
    tree_method='gpu_hist',

)
"""

In [ ]:
# Let's compute SHAP values for the last test set after the final cross-validation loop
explainer = shap.TreeExplainer(clf.fit(X,y))

In [ ]:
shap_values = explainer.shap_values(X)

In [ ]:
import matplotlib
#matplotlib.use('Agg')

import shap
import matplotlib.pyplot as plt
import shap
import matplotlib.pyplot as plt

# Set up a figure
fig, ax = plt.subplots(figsize=(10, 6))

# Generate the SHAP plot
shap.summary_plot(shap_values, df.iloc[:,:-1], plot_type="bar", max_display=30)

# Save and display
plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Asumiendo que shap_values es tu matriz de valores SHAP y ya tienes un DataFrame de pandas df y meta definidos.
shap_df = pd.DataFrame(shap_values, columns=df.columns[:-1], index=df.index)

# Agregar la variable "panel" del DataFrame "meta" al DataFrame SHAP
shap_df["panel"] = meta['panel']

# Calcular valores SHAP absolutos para obtener importancias de características
shap_importances = np.abs(shap_df.iloc[:, :-1]).mean(axis=0)
top = 30

# Obtener los índices de las características principales basadas en sus importancias
top_feature_indices = np.argsort(shap_importances)[-top:]

# Seleccionar las características principales según sus índices
top_features = shap_df.columns[top_feature_indices]

# Derretir el DataFrame para la representación basada en tonos
melted_shap_df = pd.melt(shap_df, id_vars=["panel"], value_vars=top_features, var_name="variable")

# Ordena melted_shap_df por 'variable' y luego por 'panel' para que el orden de los plots coincida con el orden alfabético de las etiquetas
sorted_panels = sorted(melted_shap_df['panel'].unique())
melted_shap_df['panel'] = pd.Categorical(melted_shap_df['panel'], categories=sorted_panels, ordered=True)

# Ahora, ordena por variable y por el orden del panel
melted_shap_df = melted_shap_df.sort_values(by=['variable', 'panel'])

# Crear un diagrama de caja con Plotly
fig = px.box(
    melted_shap_df,
    x="value",
    y="variable",
    color="panel",
    title="Top 5 SHAP Values by Panel",
    labels={"value": "SHAP Value", "variable": "Feature", "panel": "Panel"},
    orientation="h",
    category_orders={"variable": top_features.tolist()}
)

# Agregar línea vertical en x=0
fig.add_shape(
    type='line',
    line=dict(
        dash='dash',
    ),
    x0=0,
    x1=0,
    y0=-0.5,
    y1=top - 0.5
)

fig.update_layout(
    xaxis_title="SHAP Value",
    yaxis_title="Feature",
    boxmode='group',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

# Guardar la figura en un archivo HTML
fig.write_html("shap_summary_plotly.html")



In [ ]:

import matplotlib

import shap
import matplotlib.pyplot as plt
import shap
import matplotlib.pyplot as plt

# Set up a figure
fig, ax = plt.subplots(figsize=(10, 4))

# Generate the SHAP plot
shap.summary_plot(shap_values, np.log(df.iloc[:, :-1] +1), plot_type="layered_violin", max_display=30, color='seismic', show=False)
#seismic coolwarm
# Save and display
plt.tight_layout()
plt.savefig("shap_summary_plot_layered.png", dpi=600)  # dpi can be adjusted based on desired resolution

plt.show()


In [ ]:

import matplotlib


import shap
import matplotlib.pyplot as plt
import shap
import matplotlib.pyplot as plt

# Set up a figure
fig, ax = plt.subplots(figsize=(10, 6))

# Generate the SHAP plot
shap.summary_plot(shap_values, df.iloc[:,:-1], plot_type="bar", max_display=30)

# Save and display
plt.tight_layout()
plt.show()
plt.savefig("shap_summary_plot_bar.png", dpi=600)  # dpi can be adjusted based on desired resolution

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set the style and context for the plot
sns.set_style("whitegrid")
sns.set_context("talk")

# Assuming shap_values_1 is your SHAP values matrix
# You can create a DataFrame to make it easier to work with
shap_df = pd.DataFrame(shap_values, columns=df.columns[:-1], index=df.index)

# Add the "panel" variable from the "meta" DataFrame to the SHAP DataFrame
shap_df["panel"] = meta['vital']

# Calculate absolute SHAP values to get feature importances
shap_importances = np.abs(shap_df.iloc[:, :-1]).mean(axis=0)
top = 30

# Get the indices of the top features based on their importances
top_feature_indices = np.argsort(shap_importances)[-top:]

# Select the top features based on their indices
top_features = shap_df.columns[top_feature_indices]

# Melt the DataFrame for hue-based plotting
melted_shap_df = pd.melt(shap_df, id_vars=["panel"], value_vars=top_features, var_name="variable")

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(13, 17))

# Use sns.boxplot directly on the ax object with color referenced from your provided code
sns.boxplot(
    data=melted_shap_df, x="value", y="variable", hue="panel", ax=ax
)

# Add the red dotted line at x=0
ax.axvline(0, color='red', linestyle='--', linewidth=1.5)

# Invert the Y axis so the most important features are at the top
ax.invert_yaxis()

# Customize title and labels for better aesthetics
ax.set_xlabel("SHAP Value")
ax.set_ylabel("Feature")

# Add a legend and adjust its appearance
ax.legend(title="Panel", title_fontsize='16', loc="upper right", fontsize='14')

plt.tight_layout()

# Save and display the figure
plt.savefig("shap_vital_box.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style and context for the plot
sns.set_style("whitegrid")
sns.set_context("talk")

# Suponiendo que shap_values_1 es tu matriz de valores SHAP
# Puedes crear un DataFrame para facilitar su manipulación
shap_df = pd.DataFrame(shap_values, columns=df.columns[:-1], index=df.index)

# Añadir la variable "panel" del DataFrame "meta" al DataFrame SHAP
shap_df["panel"] = meta['panel']

# Calcular valores SHAP absolutos para obtener importancias de características
shap_importances = np.abs(shap_df.iloc[:, :-1]).mean(axis=0)
top = 5
# Obtener los índices de las características principales según su importancia
top_feature_indices = np.argsort(shap_importances)[-top:]

# Seleccionar las características principales según sus índices
top_features = shap_df.columns[top_feature_indices]

# Crear un gráfico boxen más grande con colores consistentes identificando el panel
plt.figure(figsize=(10, 10))
melted_shap_df = pd.melt(shap_df, id_vars=["panel"], value_vars=top_features[::-1], var_name="variable")

sns.boxplot(
    data=melted_shap_df, x="value", y="variable", hue="panel",
    hue_order=sorted(melted_shap_df['panel'].unique()),
    orient="h", dodge=True, fliersize=1,linewidth=0.5
)

# Añadir la línea roja punteada en x=0
plt.axvline(0, color='red', linestyle='--')

# Establecer las etiquetas y el título con tamaños ajustados
plt.xlabel("SHAP Value", fontsize=12)
plt.title("SHAP Value Distribution across Panels", fontsize=14)

# Mover la leyenda fuera del área del gráfico, ajustar su tamaño y orientación
plt.legend(title="Panel", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, title_fontsize=12)

# Ajuste apretado para garantizar una visualización adecuada del gráfico
plt.tight_layout()
plt.ylabel("")
# Guardar el gráfico con una resolución alta
plt.savefig("shap_panel_box.png", dpi=600, bbox_inches='tight')

# Mostrar el gráfico
plt.show()


In [ ]:
save = pd.DataFrame(shap_values, columns= df.iloc[:,:-1].columns, index= df.iloc[:,:-1].index)

In [ ]:
save

In [ ]:
save.to_pickle("/content/drive/MyDrive/pan_cancer_diner/XGBOOST all shap files/shap_1.pkl")